In [2]:
import sys
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import argparse
import os
import glob

def filepaths(args):
    filepaths = glob.glob('*.csv')
    if args.pick is not None:
        filepaths = [i for i in filepaths if args.pick in i]
    print(filepaths)

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--pick", default=None, type=str)
    parser.add_argument("--ratio", default=None, type=list)
    return parser.parse_args()

def make_csv():
    dataframe_list = []

    for i in range(len(filepaths)):
        dataframe_list.append(pd.read_csv(filepaths[i]))


if __name__ == "__main__":
    args = parse_args()
    #filepaths(args)



usage: ipykernel_launcher.py [-h] [--pick PICK] [--ratio RATIO]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"e52193c9-d1f7-447d-b829-161b0a434155" --shell=9002 --transport="tcp" --iopub=9004 --f=/opt/ml/.local/share/jupyter/runtime/kernel-v2-105539oBHgB40Dq4jj.json


SystemExit: 2